In [2]:
import numpy as np
import tensorflow as tf

from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

from constants import *
from encoding_decoding_lookup import *

## Training our model

### Dataset

### Diacritizer

In [ ]:
# for training we need to follow these steps:
'''
1. input is from lookup table all sentences concatenated together
2. For variable length input we need sliding window and also it helps to get the contextual meaning due to overlapping
'''

In [ ]:
# just like the lab, but with tf, we would do init with all needed variables and a forward fn
#what we need: self, window_size, lstm_size, dropout_rate,embedding_size
class Diacritizer:
    def __init__(self)
    def __init__(self, vocab_size=35181, embedding_dim=50, hidden_size=50, n_classes=len(tag_map)):
    

--- 